<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Langchain/func_call_db_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Function Calling Database Agent with LangChain and Groq LLM

## Introduction
This notebook demonstrates the implementation of a function calling database agent that combines Large Language Models (LLM) with SQL database operations using LangChain and Groq's LLM services. The project focuses on analyzing employee salary data stored in a SQLite database by leveraging predefined SQL functions that can be called through natural language queries.

## Goals
1. Set up a data pipeline that loads employee salary data from a CSV file into a SQLite database
2. Implement a function calling agent that maps natural language queries to specific SQL functions
3. Create specialized database functions for common HR analytics tasks including:
   - Analyzing salary distributions across divisions and departments
   - Tracking overtime pay metrics
   - Examining gender-based employment statistics
   - Calculating longevity pay across different grade levels

The notebook uses the Llama 3.3 70B model through Groq's API to interpret natural language queries and route them to appropriate predefined SQL functions, enabling efficient and structured access to HR data through conversational interactions.


In [1]:
%pip install -qU pyodbc tabulate langchain langchain-community langchain-core langchain-experimental langchain-groq SQLAlchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [7]:
from google.colab import files
uploaded = files.upload()

Saving helpers.py to helpers.py


In [8]:
import json
from langchain_groq import ChatGroq
import pandas as pd

from sqlalchemy import create_engine
import numpy as np
from sqlalchemy import text


import helpers
from helpers import (
    get_avg_salary_and_female_count_for_division,
    get_total_overtime_pay_for_department,
    get_total_longevity_pay_for_grade,
    get_employee_count_by_gender_in_department,
    get_employees_with_overtime_above,
)


llm_name = "llama-3.3-70b-versatile"
model = ChatGroq(model=llm_name)

# for the weather function calling
from groq import Groq
client = Groq()

from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase

# create a db from csv file

# Path to your SQLite database file
database_file_path = "./db/salary.db"


# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f"sqlite:///{database_file_path}")
file_url = "/content/salaries_2023.csv"
os.makedirs(os.path.dirname(database_file_path), exist_ok=True)
df = pd.read_csv(file_url).fillna(value=0)
df.to_sql("salaries_2023", con=engine, if_exists="replace", index=False)


def run_conversation(
    query="""What is the average salary and the count of female employees
    #                   in the ABS 85 Administrative Services division?""",
):

    messages = [
        # {
        #     "role": "user",
        #     "content": """What is the average salary and the count of female employees
        #               in the ABS 85 Administrative Services division?""",
        # },
        {
            "role": "user",
            "content": query,
        },
        # {
        #     "role": "user", # gives error request too large
        #     "content": """How many employees have overtime pay above 5000?""",
        # },
    ]

    # Call the model with the conversation and available functions
    response = client.chat.completions.create(
        model=llm_name,
        messages=messages,
        tools=helpers.tools_sql,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    # print(response_message.model_dump_json(indent=2))
    # print("tool calls: ", response_message.tool_calls)

    tool_calls = response_message.tool_calls
    if tool_calls:
        # Step 3: call the function
        available_functions = {
            "get_avg_salary_and_female_count_for_division": get_avg_salary_and_female_count_for_division,
            "get_total_overtime_pay_for_department": get_total_overtime_pay_for_department,
            "get_total_longevity_pay_for_grade": get_total_longevity_pay_for_grade,
            "get_employee_count_by_gender_in_department": get_employee_count_by_gender_in_department,
            "get_employees_with_overtime_above": get_employees_with_overtime_above,
        }
        messages.append(response_message)  # extend conversation with assistant's reply

        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            if function_name == "get_employees_with_overtime_above":
                function_response = function_to_call(amount=function_args.get("amount"))
            elif function_name == "get_total_longevity_pay_for_grade":
                function_response = function_to_call(grade=function_args.get("grade"))
            else:
                function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": str(function_response),
                }
            )  # extend conversation with function responses
            second_response = client.chat.completions.create(
                model=llm_name,
                messages=messages,
            )  # get a new response from the model where it can see the function response

        return second_response


# Example calls to the functions
if __name__ == "__main__":
    res = (
        run_conversation(
            query="""What is the average base salary for female employees in the Alcohol Beverage Services department? How much overtime pay is given on average to employees in grade 21?"""
        )
        .choices[0]
        .message.content
    )

    print(res)


Based on the provided data, the average base salary for female employees in the Alcohol Beverage Services department is not available (None) and there are 0 female employees in this department. 

The average overtime pay for employees in grade 21 is $954,175.36. 

Please note that the grade number for overtime pay does not necessarily correspond to the grade number for longevity pay. For grade 21, the total longevity pay is $580,836.75.
